# Imports

In [ ]:
import torch
import torch.nn as nn
from utils.datautils import *
from utils.MLutils import *
from utils.resources import *
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import BertModel
from sklearn.metrics import classification_report
from data.variables import *

# Procesamiento

In [2]:
linux = True
device = None

if linux:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
else:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("usando:", device)

usando: cuda


## Busqueda de fuentes

- Fuente 1: Conjunto de preguntas en espa;ol
- Fuente 2: Dataset provisto para Notebook 10
- Fuente 3: Dataset sintetico generado con Gemini
- Fuente 4: Articulos de Wikipedia
- Fuente 5: Subtitulos de peliculas
- Fuente 6: Mixture of preguntas y afirmaciones

In [3]:
questions, question_for_mixture = get_questions()
oraciones_rnn = get_notebook_dataset()
oraciones_sinteticas = get_gemini_dataset()
frases_wikipedia = get_wikipedia_dataset()
esperando_la_carroza, frases_relatos_salvajes = get_pelis_dataset()
mixtures = get_mixture_dataset(oraciones_sinteticas, question_for_mixture)

Se descargaron 5000 preguntas en Español.
Se descargaron 997 oraciones en Español (del dataset del notebook 10).
Hay 1413 oraciones sintéticas.
Se cargaron 6648 frases de Wikipedia.
✅ Se extrajeron 947 frases completas y se guardaron en 'dialogos_esperando_la_carroza.json'
Frases extraídas en total: 947
✅ Se extrajeron 1000 frases de Relatos Salvajes.


## Juntamos las fuentes

In [4]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes + mixtures

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))
print('Cantidad de oraciones Compuestas:',len(mixtures))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Cantidad total de oraciones: 20244
Cantidad de oraciones de preguntas: 5000
Cantidad de oraciones en espa;ol de hugging face: 997
Cantidad de oraciones sintéticas: 1413
Cantidad de oraciones de Wikipedia: 6648
Cantidad de oraciones de Esperando la carroza: 947
Cantidad de oraciones de Relatos Salvajes: 1000
Cantidad de oraciones Compuestas: 4239
Algunas oraciones aleatorias:


['¿Cuál fue el resultado del encuentro?',
 'Con mi suegra.',
 'Mi operación requería mucho reposo, así que vi las temporadas 1 a 3 de “Anatomía según Grey”. Me ayudó que McDreamy fuera neurocirujano. Yo prestaba atención a cómo colocaban al paciente básicamente por intentar adivinar lo que habían hecho conmigo. Desde mi cirugía en 2018, vi más representaciones de neurocirugía en Hospital Playlist que en cualquier otro programa médico que haya visto desde Grey. Es agradable tener arte y entretenimiento que nos ayude a darle sentido a nuestra vida.',
 'El estrecho de Bering fue un puente de tierra en la Edad de Hielo. ¿A cuántos narcos pudieron detener el octubre pasado?',
 '¿Cómo se suele realizar la separación de la leche en el cuajado? El nuevo Ford Puma es un coche híbrido.']

Separamos en conjuntos de `train` y `test` con el tokenizer de `BERT`

In [5]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)

print(len(train_sents))
print(len(test_sents))

19231
1013


## Importamos el modelo

### Sin atencion

In [ ]:
from train.RNNBidirectional import PunctuationCapitalizationRNNBidirectional

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

for param in bert_model.parameters():
    param.requires_grad = False

N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in bert_model.pooler.parameters():
    param.requires_grad = True


model = PunctuationCapitalizationRNNBidirectional(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)

ckpt = torch.load("model_bidirec.pt", map_location=device)
# si guardaste state_dict puro
if isinstance(ckpt, dict) and "model_state_dict" not in ckpt:
    model.load_state_dict(ckpt)

# si guardaste un dict con más cosas (epoch, optim, etc.)
elif "model_state_dict" in ckpt:
    model.load_state_dict(ckpt["model_state_dict"])

Total parameters: 180,944,905
Trainable parameters: 17,857,801


In [ ]:
evaluate(model, dataloader_test, device)


Classification Report for Punctuation Prediction:
              precision    recall  f1-score   support

           Ø       0.99      0.98      0.98     19951
           ,       0.52      0.85      0.64       499
           .       0.83      0.76      0.79       652
           ?       0.84      0.70      0.76       565
           ¿       0.93      0.85      0.89       579

    accuracy                           0.96     22246
   macro avg       0.82      0.83      0.81     22246
weighted avg       0.97      0.96      0.96     22246


Classification Report for Capitalization Prediction:
              precision    recall  f1-score   support

       lower       0.98      0.99      0.99     15161
        init       0.96      0.95      0.95      4734
         mix       0.90      0.66      0.76        65
       upper       0.87      0.95      0.91       260

    accuracy                           0.98     20220
   macro avg       0.93      0.89      0.90     20220
weighted avg       0.98   

### Con atencion

In [ ]:
from train.RNNBidirectionalAttention import PunctuationCapitalizationRNNBidirectionalAttention 

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

for param in bert_model.parameters():
    param.requires_grad = False

N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in bert_model.pooler.parameters():
    param.requires_grad = True


model = PunctuationCapitalizationRNNBidirectionalAttention(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)

Total parameters: 181,995,529
Trainable parameters: 18,908,425


In [ ]:
punct_weights_tensor, cap_weights_tensor = compute_class_weights(
    dataloader_train,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS),
    device=device,
    beta=0.7
)

criterion_punct = nn.CrossEntropyLoss(ignore_index=-100, weight=punct_weights_tensor)
criterion_cap   = nn.CrossEntropyLoss(ignore_index=-100, weight=cap_weights_tensor)


trainable_params_attn = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.lstm1.parameters()) \
  + list(model.lstm2.parameters()) \
  + list(model.attention.parameters()) \
  + list(model.punct_classifier.parameters()) \
  + list(model.cap_classifier.parameters())

optimizer = torch.optim.AdamW(trainable_params_attn, lr=2e-5)

Epoch 1 | Train Loss: 1.9828
Epoch 2 | Train Loss: 1.2857
Epoch 3 | Train Loss: 0.8370
Epoch 4 | Train Loss: 0.6504
Epoch 5 | Train Loss: 0.5590
Epoch 6 | Train Loss: 0.5034
Epoch 7 | Train Loss: 0.4591
Epoch 8 | Train Loss: 0.4270
Epoch 9 | Train Loss: 0.4025
Epoch 10 | Train Loss: 0.3850
Epoch 11 | Train Loss: 0.3628
Epoch 12 | Train Loss: 0.3470
Epoch 13 | Train Loss: 0.3322
Epoch 14 | Train Loss: 0.3163
Epoch 15 | Train Loss: 0.3061
Epoch 16 | Train Loss: 0.2918
Epoch 17 | Train Loss: 0.2832
Epoch 18 | Train Loss: 0.2715
Epoch 19 | Train Loss: 0.2644
Epoch 20 | Train Loss: 0.2529


In [ ]:
train(
    model, 
    dataloader_train=dataloader_train, 
    optimizer=optimizer, 
    criterion_punct=criterion_punct, 
    criterion_cap = criterion_cap, 
    device=device, 
    epochs=20
)

## Evaluacion

In [ ]:
evaluate(model, dataloader_test, device)

In [29]:
entrada = "es terrible lo que está pasando en chaco te enteraste"
print(f"{predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

Es terrible, lo que está pasando, en Chaco, te enteraste.


In [ ]:
predicciones_TP(
    model=model,
    tokenizer=tokenizer,
    device=device,
    csv_path="data/testeo.csv",
)

## Export modelo

In [14]:
torch.save(model.state_dict(), "model_bidirec_attention.pt")